# Brain Tumor Detection

### Install the dataset from Kaggle

In [31]:
! pip install kaggle

In [33]:
! kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors



  0%|          | 0.00/15.1M [00:00<?, ?B/s]
  7%|6         | 1.00M/15.1M [00:00<00:13, 1.13MB/s]
 13%|#3        | 2.00M/15.1M [00:01<00:10, 1.33MB/s]
 20%|#9        | 3.00M/15.1M [00:02<00:08, 1.55MB/s]
 26%|##6       | 4.00M/15.1M [00:02<00:07, 1.61MB/s]
 33%|###3      | 5.00M/15.1M [00:03<00:06, 1.57MB/s]
 40%|###9      | 6.00M/15.1M [00:04<00:06, 1.55MB/s]
 46%|####6     | 7.00M/15.1M [00:05<00:06, 1.37MB/s]
 53%|#####2    | 8.00M/15.1M [00:06<00:05, 1.30MB/s]
 60%|#####9    | 9.00M/15.1M [00:06<00:04, 1.38MB/s]
 66%|######6   | 10.0M/15.1M [00:07<00:03, 1.48MB/s]
 73%|#######2  | 11.0M/15.1M [00:08<00:03, 1.32MB/s]
 79%|#######9  | 12.0M/15.1M [00:08<00:02, 1.36MB/s]
 86%|########6 | 13.0M/15.1M [00:09<00:01, 1.33MB/s]
 93%|#########2| 14.0M/15.1M [00:10<00:00, 1.39MB/s]
 99%|#########9| 15.0M/15.1M [00:11<00:00, 1.17MB/s]
100%|##########| 15.1M/15.1M [00:11<00:00, 1.34MB/s]


In [36]:
import zipfile

with zipfile.ZipFile("brain-mri-images-for-brain-tumor-detection.zip", "r") as zip_ref:
    zip_ref.extractall(".")

In [13]:
import pickle

from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix

from COV_brain_tumor_detection import (
    create_data_generators,
    TEST_DIR,
    TRAIN_DIR,
    VAL_DIR,
    create_model,
    preprocess_images,
    load_data,
    split_data,
    clean_directory,
    remove_directories,
    save_new_images,
    plot_confusion_matrix,
    plot_data_distribution,
    plot_model_performance,
    plot_samples,
)

In [ ]:
remove_directories([TRAIN_DIR, TEST_DIR, VAL_DIR])

In [ ]:
split_data("../brain_tumor_dataset")

In [ ]:
plot_data_distribution()

In [ ]:
X_train, y_train, labels = load_data(TRAIN_DIR)
X_test, y_test, _ = load_data(TEST_DIR)
X_val, y_val, _ = load_data(VAL_DIR)

In [ ]:
plot_samples(X_train, y_train, labels)

In [ ]:
X_train_prep = preprocess_images(X_train)
X_val_prep = preprocess_images(X_val)
X_test_prep = preprocess_images(X_test)

In [ ]:
plot_samples(X_train_prep, y_train, labels)

In [ ]:
save_new_images(X_train_prep, y_train, folder_name=TRAIN_DIR)
save_new_images(X_val_prep, y_val, folder_name=VAL_DIR)
save_new_images(X_test_prep, y_test, folder_name=TEST_DIR)

In [ ]:
clean_directory(TRAIN_DIR)
clean_directory(VAL_DIR)
clean_directory(TEST_DIR)

In [ ]:
model = create_model()
model.summary()

In [ ]:
train_generator, validation_generator = create_data_generators()

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=7,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=2,
    callbacks=[EarlyStopping(monitor="val_accuracy", mode="max", patience=6)],
)

model.save('cnn-vgg16.h5')

with open("model_history.pkl", "wb") as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
with open("model_history.pkl", "rb") as file_pi:
    loaded_history = pickle.load(file_pi)
    
plot_model_performance(loaded_history)

In [ ]:
predictions_test = [1 if x > 0.5 else 0 for x in model.predict(X_test_prep)]

In [ ]:
accuracy = accuracy_score(y_test, predictions_test)
print("Test Accuracy = %.2f" % accuracy)

In [ ]:
confusion_mtx = confusion_matrix(y_test, predictions_test)
plot_confusion_matrix(confusion_mtx, classes=list(labels.items()), normalize=False)

In [ ]:
import numpy as np

from PIL import Image
from keras.saving.save import load_model

In [15]:
model = load_model('COV_brain_tumor_detection/cnn-vgg16.h5')

In [25]:
def predict_tumor(img_path):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_tensor = np.expand_dims(img_array, axis=0)
    img_tensor = img_tensor.astype('float32') / 255.0

    prediction = model.predict(img_tensor)
    
    if prediction[0][0] > 0.5:
        return "The tumour was found"
    else:
        return "No tumour was found"

In [26]:
image_path = 'brain_tumor_dataset/no/2 no.jpeg'

In [ ]:
result = predict_tumor(image_path)
print(result)